In [ ]:
%gui qt
import sys
import numpy as np
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.Point import Point

In [ ]:
def PYSIGLAB(data):
    """
        SIGLAB : Born out of frustration with the you-know-who
    """
    app_created = False
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtGui.QApplication(sys.argv)
        app_created = True
    app.references = set()
    
    
    w = QtGui.QWidget()
    w.setWindowTitle('SIGLAB')
    w.resize(800, 600)
    
    ## Create a grid layout to manage the widgets size and position
    layout = QtGui.QGridLayout()
    w.setLayout(layout)
    w.show()
    
    #generate layout
    view = pg.GraphicsLayoutWidget()
    
    p1 = view.addPlot(row=1, col=0)
    p2 = view.addPlot(row=2, col=0)

    region = pg.LinearRegionItem(brush=(0, 255, 0, 127))
    region.setZValue(10)
    
    p2.addItem(region, ignoreBounds=True)
    p1.setAutoVisible(y=True)
    
    b1 = QtGui.QPushButton('Clean')
    b2 = QtGui.QPushButton('Save')
    
    ## Add widgets to the layout
    layout.addWidget(view, 0, 0, 1, 2)
    layout.addWidget(b1, 1, 0)   
    layout.addWidget(b2, 1, 1)   

    p1.plot(data, pen="b")

    p2.plot(data, pen="w")

    def update():
        region.setZValue(20)
        minX, maxX = region.getRegion()
        p1.setXRange(minX, maxX, padding=0)

    region.sigRegionChanged.connect(update)

    def updateRegion(window, viewRange):
        rgn = viewRange[0]
        region.setRegion(rgn)

    p1.sigRangeChanged.connect(updateRegion)

    region.setRegion([1000, 2000])
    
    #cross hair
    vLine = pg.InfiniteLine(angle=90, movable=False)
    hLine = pg.InfiniteLine(angle=0, movable=False)
    p1.addItem(vLine, ignoreBounds=True)
    p1.addItem(hLine, ignoreBounds=True)
    
    def b1_clicked(evt):
        print "you clicked Clear"
    
    b1.clicked.connect(b1_clicked)
    
    def b2_clicked(evt):
        print "you clicked Save"
        
    b2.clicked.connect(b2_clicked)
    
    vb = p1.vb
    
    
    def mouseMoved(evt):
        pos = evt[0]  ## using signal proxy turns original arguments into a tuple
        if p1.sceneBoundingRect().contains(pos):
            mousePoint = vb.mapSceneToView(pos)
            index = int(mousePoint.x())
            if index > 0 and index < len(data1):
                label.setText("<span style='font-size: 12pt'>x=%0.1f,<span style='color: red'>y1=%0.1f</span>,<span style='color: green'>y2=%0.1f</span>"
                              %(mousePoint.x(), data1[index], data2[index]))
            vLine.setPos(mousePoint.x())
            hLine.setPos(mousePoint.y())


    proxy = pg.SignalProxy(p1.scene().sigMouseMoved, rateLimit=60, slot=mouseMoved)
    
    if app_created:
        app.exec_()
    return w

In [ ]:
data = 10000 + 15000 * pg.gaussianFilter(np.random.random(size=10000), 10) + 3000 * np.random.random(size=10000)

win = PYSIGLAB(data)